In [1]:
%run _standard_imports.ipynb

python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [2]:
import stat
# from sh import ssh
# farm3 = ssh.bake("farm3-login")
bsub = sh.Command('bsub')

In [44]:
pf_40_freeze_manifest_fn = '/nfs/team112_internal/production_files/Pf/4_0/pf_40_freeze_manifest.tab'
samples_subset_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PfCP_4.txt' # From Roberto March 30, 2106 14:14
cinzia_metadata_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PF_metadata_base.csv' # From Cinzia 22/03/2016 07:47
cinzia_extra_metadata_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PF_extrametadata.csv' # From Cinzia 22/03/2016 08:22
compiled_metadata_3488_fn = '/nfs/team112_internal/rp7/data/Pf/4_0/meta/PfCP_4_3488_withtypes.txt'
output_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/pysamstats'
# output_dir = '/nfs/team112_internal/rp7/data/Pf/4_0/pysamstats'
# !mkdir -p {"%s/scripts" % output_dir}
# !mkdir -p {"%s/results" % output_dir}

analysis_chroms = collections.OrderedDict()
analysis_chroms['Pf3D7_05_v3'] = [37901, 1321390]
analysis_chroms['Pf3D7_08_v3'] = [1, 1472805]
analysis_chroms['Pf3D7_09_v3'] = [79101, 1473560]
analysis_chroms['Pf3D7_13_v3'] = [1, 2925236]

analysis_exons = collections.OrderedDict()
analysis_exons['PF3D7_0831800_exon1'] = ['Pf3D7_08_v3', 1375231, 1375299]
analysis_exons['PF3D7_0831800_exon2'] = ['Pf3D7_08_v3', 1374236, 1375084]
analysis_exons['PF3D7_0831700_exon1'] = ['Pf3D7_08_v3', 1365467, 1367506]
analysis_exons['PF3D7_0831900_exon1'] = ['Pf3D7_08_v3', 1380834, 1380971]
analysis_exons['PF3D7_0831900_exon2'] = ['Pf3D7_08_v3', 1381108, 1381835]
analysis_exons['PF3D7_0832000_exon1'] = ['Pf3D7_08_v3', 1384330, 1384377]
analysis_exons['PF3D7_0832000_exon2'] = ['Pf3D7_08_v3', 1383349, 1384212]
analysis_exons['PF3D7_1372200_exon1'] = ['Pf3D7_13_v3', 2841635, 2841703]
analysis_exons['PF3D7_1372200_exon2'] = ['Pf3D7_13_v3', 2840727, 2841485]
analysis_exons['PF3D7_1372100_exon1'] = ['Pf3D7_13_v3', 2837053, 2837136]
analysis_exons['PF3D7_1372100_exon2'] = ['Pf3D7_13_v3', 2837313, 2839058]
analysis_exons['PF3D7_1372300_exon1'] = ['Pf3D7_13_v3', 2845767, 2845850]
analysis_exons['PF3D7_1372300_exon2'] = ['Pf3D7_13_v3', 2846002, 2846538]
analysis_exons['PF3D7_0523000_exon1'] = ['Pf3D7_05_v3', 957890, 962149]


In [17]:
tbl_samples_subset = etl.fromtsv(samples_subset_fn).pushheader(['sample']).addfield('temp', 0)
print(len(tbl_samples_subset.data()))
tbl_samples_subset

3488


sample,temp
PA0007-C,0
PA0008-C,0
PA0011-C,0
PA0012-C,0
PA0015-C,0


In [42]:
tbl_type = etl.fromcsv(cinzia_extra_metadata_fn, encoding='latin1').cut(['oxford_code', 'type', 'ostype'])
tbl_type.head(1892).tail(1).displayall()

oxford_code,type,ostype
PH0047-Cx,dna,NULL


In [43]:
tbl_pf_40_freeze_manifest = (etl
    .fromtsv(pf_40_freeze_manifest_fn)
    .sub('path', 'scratch108', 'scratch109')
    .join(tbl_samples_subset, key='sample')
    .leftjoin(tbl_type, lkey='sample', rkey='oxford_code')
)
print(len(tbl_pf_40_freeze_manifest.data()))
tbl_pf_40_freeze_manifest.display(index_header=True)

3488


0|path,1|study,2|sample,3|bases,4|bases_mapped,5|avg_read_length,6|bases_of_1X_coverage,7|bases_of_5X_coverage,8|bases_of_10X_coverage,9|bases_of_50X_coverage,10|mean_coverage,11|sample_acc,12|run_acc,13|temp,14|type,15|ostype
/lustre/scratch109/malaria/pfalciparum/output/2/d/d/a/42721/1_bam_merge/pe.1.bam,PF6,PA0007-C,2074610304,1849037212,76,22703916,20226914,17341168,9063212,74.27,ERS010116,ERR018933,0,dna,Non-cultured Blood
/lustre/scratch109/malaria/pfalciparum/output/0/2/f/9/42722/1_bam_merge/pe.1.bam,PF6,PA0008-C,2529943176,2302964844,76,23160225,22873797,22241258,14633143,96.81,ERS010126,ERR018936,0,dna,Non-cultured Blood
/lustre/scratch109/malaria/pfalciparum/output/3/d/9/5/42723/1_bam_merge/pe.1.bam,PF6,PA0011-C,3707830088,3451499872,76,22960209,22798857,22697202,21815652,149.59,ERS010043,ERR015410,0,dna,Non-cultured Blood
/lustre/scratch109/malaria/pfalciparum/output/5/6/7/1/42724/1_bam_merge/pe.1.bam,PF6,PA0012-C,2910823712,2219976340,76,22982697,22430685,21499677,14970548,95.02,ERS010127,ERR021977,0,dna,Non-cultured Blood
/lustre/scratch109/malaria/pfalciparum/output/b/a/0/5/42725/1_bam_merge/pe.1.bam,PF6,PA0013-C,1706925600,1536128300,100,22819992,22447606,21882221,14505222,65.28,ERS157495,ERR211480,0,dna,Non-cultured Blood


In [45]:
tbl_pf_40_freeze_manifest.totsv(compiled_metadata_3488_fn)

In [46]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'PV0097-C')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/1/2/4/7/45344/1_bam_merge/pe.1.bam,ARCE,PV0097-C,261885800,194178900,100,22388905,17846219,8889398,12870,8.67,ERS086908,ERR126469,0,dna,NULL


In [47]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'PH0416-C')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/3/1/1/a/44438/1_bam_merge/pe.1.bam,ARC3,PH0416-C,431737500,390882450,75,22843242,21965447,19530322,106194,17.11,ERS032056,ERR039983,0,dna,NULL


In [48]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'PH0395-C')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/4/1/3/3/44417/1_bam_merge/pe.1.bam,ARC3,PH0395-C,243637800,204407475,75,22501655,18740810,9758607,19203,9.08,ERS032073,ERR040000,0,dna,NULL


In [49]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'PV0274-C')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/4/0/e/d/45490/1_bam_merge/pe.1.bam,TRAC1,PV0274-C,2228748200,2058050500,100,22819514,22553897,22145792,16271860,89.43,ERS143473,ERR180071,0,dna,NULL


In [58]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'PH0153-CW')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/0/0/4/b/44220/1_bam_merge/pe.1.bam,ARC3,PH0153-CW,1397387250,1289171925,75,22889980,22430053,21956548,12339237,56.32,ERS024152,ERR036594,0,MDA,NULL


In [59]:
tbl_pf_40_freeze_manifest.selecteq('sample', 'QE0455-C')

path,study,sample,bases,bases_mapped,avg_read_length,bases_of_1X_coverage,bases_of_5X_coverage,bases_of_10X_coverage,bases_of_50X_coverage,mean_coverage,sample_acc,run_acc,temp,type,ostype
/lustre/scratch109/malaria/pfalciparum/output/5/7/5/8/45741/1_bam_merge/pe.1.bam,TRAC1,QE0455-C,1793021400,1627002600,100,22888264,22672815,22430448,16538494,70.50,ERS174616,ERR216573,0,dna,NULL


In [50]:
len(tbl_pf_40_freeze_manifest.data())

3488

In [54]:
from Bio.Seq import Seq
print(Seq('ATATAATTCAGGGTTTAGGGTTTA').reverse_complement())

TAAACCCTAAACCCTGAATTATAT


In [52]:
# PH0395-C
print(Seq('ATATAATTCAGGGTTTAGGGTTTA').reverse_complement())

import stat
bsub = sh.Command('bsub')

breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

breakpoint_sequence = "'ATATAATTCAGGGTTTAGGGTTTA|TAAACCCTAAACCCTGAATTATAT'"
for rec in tbl_pf_40_freeze_manifest.data():
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [60]:
print(Seq('GTTCATCTGTATATAATTCAGGTTTTAGGTTTTA').reverse_complement())
print(Seq('GTTCATCTGTATATAATTCAGGGTTTAGGGTTTA').reverse_complement())


TAAAACCTAAAACCTGAATTATATACAGATGAAC
TAAACCCTAAACCCTGAATTATATACAGATGAAC


In [61]:
# PH0395-C
print(Seq('GTTCATCTGTATATAATTCAGGTTTTAGGTTTTA').reverse_complement())
print(Seq('GTTCATCTGTATATAATTCAGGGTTTAGGGTTTA').reverse_complement())

import stat
bsub = sh.Command('bsub')

breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953_34'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

breakpoint_sequence = "'GTTCATCTGTATATAATTCAGG[T|G]TTTAGG[T|G]TTTA|TAAA[A|C]CCTAAA[A|C]CCTGAATTATATACAGATGAAC'"
for rec in tbl_pf_40_freeze_manifest.data():
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


TAAAACCTAAAACCTGAATTATATACAGATGAAC
TAAACCCTAAACCCTGAATTATATACAGATGAAC
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
l /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953/results/* | wc -l
bjobs | wc -l
!egrep -v '^0$' /lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2830953/results/*

In [56]:
print(Seq('TTGGTAGGTTTAGGGTTCAGGGTT').reverse_complement())

AACCCTGAACCCTAAACCTACCAA


In [57]:
# PV0097-C
print(Seq('TTGGTAGGTTTAGGGTTCAGGGTT').reverse_complement())

import stat
bsub = sh.Command('bsub')

breakpoint_reads_dir = '/lustre/scratch109/malaria/rp7/data/Pf/4_0/breakpoint_spanning_reads/Pf3D7_13_v3_2837144'
!mkdir -p {breakpoint_reads_dir}/results
!mkdir -p {breakpoint_reads_dir}/scripts
!mkdir -p {breakpoint_reads_dir}/logs

breakpoint_sequence = "'TTGGTAGGTTTAGGGTTCAGGGTT|AACCCTGAACCCTAAACCTACCAA'"
for rec in tbl_pf_40_freeze_manifest.data():
    bam_fn = rec[0]
    ox_code = rec[2].replace('-', '_')
    print('.', end='')
    num_breakpoint_reads_fn = "%s/results/num_breakpoint_reads_%s.txt" % (breakpoint_reads_dir, ox_code)
    if not os.path.exists(num_breakpoint_reads_fn):
        script_fn = "%s/scripts/nbpr_%s.sh" % (breakpoint_reads_dir, ox_code)
        fo = open(script_fn, 'w')
        print('samtools view %s | grep -E %s | wc -l > %s' % (bam_fn, breakpoint_sequence, num_breakpoint_reads_fn), file = fo)
        fo.close()
        st = os.stat(script_fn)
        os.chmod(script_fn, st.st_mode | stat.S_IEXEC)
        bsub(
            '-G', 'malaria-dk',
            '-P', 'malaria-dk',
            '-q', 'normal',
            '-o', '%s/logs/nbpr_%s.out' % (breakpoint_reads_dir, ox_code),
            '-e', '%s/logs/nbpr_%s.err' % (breakpoint_reads_dir, ox_code),
            '-J', 'nbpr_%s' % (ox_code),
            '-R', "'select[mem>4000] rusage[mem=4000]'",
            '-M', '4000',
            script_fn
        )


AACCCTGAACCCTAAACCTACCAA
...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................